In [ ]:
import os

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import config

import presets
import create_weather_data_catalogue as cwdc
import aggregate_tifs_to_df as at2d

In [ ]:
# cropmask_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'cereals.tif')
# interp_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'temporarycrops.tif')


cropmask_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'geoglam_wc_maizemask.tif')
interp_tif_filepath = os.path.join(config.FOLDERPATH_MASKS, 'geoglam_wc_interp.tif')

In [ ]:
selected_year = 2019 # options: 2019, 2020, 2021, 2022, 2023

In [ ]:
YEARS = [selected_year]

In [ ]:
os.makedirs(config.FOLDERPATH_AGGREGRATE_DFS, exist_ok=True)

In [ ]:
xy_gdf = at2d.create_xy_gdf(
    cropmask_tif_filepath = cropmask_tif_filepath,
    interp_tif_filepath = interp_tif_filepath,
)

In [ ]:
xy_gdf.to_file(os.path.join(
    config.FOLDERPATH_AGGREGRATE_DFS,
    'xy.geojson',
))

In [ ]:
weather_catalogue_df = cwdc.create_weather_data_catalogue_df(
    years = YEARS,
    attribute_settings_dict = {
        presets.ATTR_CHIRPS: cwdc.Settings(
            attribute_folderpath = config.FOLDERPATH_CLUSTERFILES_CHIRPS,
            download_folderpath = config.FOLDERPATH_CHC_CHIRP_V2P0_P05,
        ),
        presets.ATTR_CPCTMAX: cwdc.Settings(
            attribute_folderpath = config.FOLDERPATH_CLUSTERFILES_CPCTMAX,
        ),
        presets.ATTR_CPCTMIN: cwdc.Settings(
            attribute_folderpath = config.FOLDERPATH_CLUSTERFILES_CPCTMIN,
        ),
        presets.ATTR_ESI4WK: cwdc.Settings(
            attribute_folderpath = config.FOLDERPATH_CLUSTERFILES_ESI4WK,
        ),
        presets.ATTR_NSIDC: cwdc.Settings(
            attribute_folderpath = config.FOLDERPATH_CLUSTERFILES_NSIDC,
        ),
        presets.ATTR_NDVI_INTERP: cwdc.Settings(
            attribute_folderpath = os.path.join(config.FOLDERPATH_OUTPUTS, 'interp_out/ndvi/tst_interp'),
        ),
    }
)

In [ ]:
weather_catalogue_df

In [ ]:
weather_catalogue_df[at2d.METHOD_COL] \
= at2d.LoadTIFMethod.READ_AND_CROP
weather_catalogue_df.loc[
    weather_catalogue_df[cwdc.FILETYPE_COL] == cwdc.TIF_GZ_EXT,
    at2d.METHOD_COL
] = at2d.LoadTIFMethod.COREGISTER_AND_CROP
weather_catalogue_df.loc[
    weather_catalogue_df[cwdc.ATTRIBUTE_COL] == presets.ATTR_NDVI_INTERP, 
    at2d.METHOD_COL
] = at2d.LoadTIFMethod.READ_NO_CROP

In [ ]:
aggregated_df = at2d.aggregate_tifs_to_df(
    catalogue_df = weather_catalogue_df,
    mask_tif_filepaths = [
        cropmask_tif_filepath,
        interp_tif_filepath,
    ],
    roi_geom_filepath = config.FILEPATH_ROI_SHAPE,
    ref_tif_filepath = config.FILEPATH_REFERENCE_GEOTIFF,
    csvs_folderpath = config.FOLDERPATH_CSVS,
)

In [ ]:
aggregated_df

In [ ]:
aggregated_df.describe()

In [ ]:
aggregated_df.to_pickle(os.path.join(
    config.FOLDERPATH_AGGREGRATE_DFS,
    f'{config.JOB_NAME}_{selected_year}.pickle',
))